# Project 2: UFO Sightings in Wyoming and Yellowstone National Park Visitation

## Group 4: Jacob Douthett and Sam Espe

### Overview

The goal of this project is to practice implementing the Extract-Transform-Load (ETL) process. We chose to use data on reports of UFO sightings (https://www.kaggle.com/datasets/NUFORC/ufo-sightings?resource=download) and the yearly visitor count at Yellowstone National Park in Wyoming (https://data.world/nps/yellowstone-np). We limited the UFO sightings to those within the state of Wyoming to create a comparable data set. To perform the ETL process, we used Pandas in a Jupyter Notebook, PostgreSQL as the database framework, and modules from SQLAlchemy to connect the two entities. 

### Extract

We start by importing the dependencies we need to complete the project. We start by importing the Pandas library| (using the common alias `pd`), as this will be our main tool to reduce, clean, and reorganize our data to load into our database. We also import `create_engine` and `inspect` from SQLAlchemy, as we will need these methods to communicate with our PostgreSQL database. Finally, we import information from the `config.py` file to construct the connection string.

In [1]:
# Import dependencies

import pandas as pd
from sqlalchemy import create_engine, inspect
from config import postgres_db_name, postgres_password, postgres_port, postgres_host

Next, we use Pandas to read the data from the CSV files (located in the `Resources` folder) into dataframes. The UFO data is loaded into `ufo_df` and the Yellowstone National Park visitation data is loaded into `yellowstone_df`. We used `.head()` on each dataframe to preview the data, and to verify that it loaded correctly. 

The Jupyter Notebook returns a warning for the UFO dataframe that columns 5 and 9 have mixed data types. Column 9 (`latitude`) is not data that we are interested in for the final database, so the mixed data type is irrelevant. Column 5 (`duration (seconds)`) will be part of the final database, and we address the data type mismatch later in the notebook.

In [2]:
# Extract data into dataframes

ufo_file = "Resources/ufo_data_scrubbed.csv"
ufo_df = pd.read_csv(ufo_file)

ufo_df.head()

C:\Users\itssm\AppData\Local\Temp\ipykernel_22096\911020618.py:4: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  ufo_df = pd.read_csv(ufo_file)


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [3]:
yellowstone_file = "Resources/Yellowstone_Annual_Park_Recreation_Visitation_1904-2018.csv"
yellowstone_df = pd.read_csv(yellowstone_file)

yellowstone_df.head()

,year,recreationvisitors,totalrecreationvisitors
0,1904,"13,727","183,271,650"
1,1905,"26,188","183,271,650"
2,1906,"17,182","183,271,650"
3,1907,"16,414","183,271,650"
4,1908,"19,542","183,271,650"


### Transform

Our next step is to transform the data. For the UFO data, we want to end up with a dataframe of complete entries (no NaNs) that only contains UFO reports from Wyoming, USA. For the Yellowstone National Park visitation data, we want to end up with a database that contains years and the corresponding number of visitors for that year.

The UFO data set requires more work to refine it, so we start with that.

#### Transform UFO data

We start by removing any entries from the worldwide data set that contain any NaN values. This will ensure that our end product contains complete data for each report. 

In [4]:
# Drop entries that have NaN values

ufo_no_nan_states = ufo_df.dropna(axis=0, how='any')

We are only interested in reports that come from Wyoming, so that we can compare the frequency of UFO reports in Wyoming to the visitor counts to Yellowstone National Park, located in Wyoming. The data set abbreviates Wyoming to 'wy', so we use Pandas to filter the dataframe to only include reports where the state is 'wy'. 

In [5]:
# Filter UFO dataframe to include only entries from Wyoming

ufos_wyoming_df = ufo_no_nan_states[ufo_no_nan_states["state"].str.contains("wy")]

In working with the data within the `datetime` column, we discovered that two records had invalid datetimes, which prevented us from continuing. The problem with both entries was that the time was recorded as "24:00". Pandas only recognizes hours from 0 to 23, and that caused the code not to work. We made the decision to remove the two offending records, since it seemed improper to edit the data itself.

In [6]:
# Dropping indices #6496 and #37861 because the datetime objects are invalid

ufos_wyoming_df = ufos_wyoming_df.drop(index=6496)
ufos_wyoming_df = ufos_wyoming_df.drop(index=37861)

To compare the UFO reports to the Yellowstone National Park visitation data, we created a new column that contained the year of the UFO sighting report. This way, an analyst could easily correlate reports to the visitation data of that year.

In [7]:
# Extract year from datetime object, save it to its own series

ufos_wyoming_df['year'] = pd.DatetimeIndex(ufos_wyoming_df['datetime']).year

For the final database, we did not want to include all of the data for each entry that the original data set provided. We decided only to include the year of the report, the city and state of the sighting, the duration of the event in seconds, the shape of the UFO, and the observer's comments. To obtain a dataframe with only the desired information, we dropped the rest of the columns.

In [8]:
# Filter columns of UFO dataframe to include only: Year, City, State, Duration (seconds), Shape, and Comments

ufos_wy_for_db = ufos_wyoming_df.copy()

ufos_wy_for_db = ufos_wy_for_db.drop(labels=["datetime", "country", "duration (hours/min)", "date posted", "latitude", "longitude "], axis = 1)

We renamed some of the columns to clarify their meaning as well as to make them conform to proper naming conventions.

In [9]:
# Rename columns

ufos_wy_for_db = ufos_wy_for_db.rename(columns={"duration (seconds)":"duration_seconds", "shape":"ufo_shape", "comments":"sighting_description"})

Here, we fix the data type of the `duration_seconds` column. There was originally a combination of integers and floating point numbers. We cast all of the data in the column as floating point numbers, so we did not lose any of the original information.

In [10]:
# Fix duration data type — make the whole series floating points instead of strings.

ufos_wy_for_db.duration_seconds = ufos_wy_for_db.duration_seconds.astype(float)

#### Transform Yellowstone Data

In [11]:
# Filter columns of Yellowstone dataframe to include only Year and Recreation Visitors

yellowstone_df = yellowstone_df.drop(labels = "totalrecreationvisitors", axis = 1)

In [12]:
# Rename recreationvisitors column

yellowstone_df = yellowstone_df.rename(columns={"recreationvisitors":"visitor_count"})

In [13]:
# Remove commas from visitor_counts so we can turn into integer

yellowstone_df.visitor_count = yellowstone_df.visitor_count.str.replace(",", "")

In [14]:
# Change data type of visitor_count column from string to integer

yellowstone_df.visitor_count = yellowstone_df.visitor_count.astype(int)

### Load

#### Connect to PostgreSQL database

In [15]:
# Save data from config.py as variables to use in connection string
password = postgres_password
port = postgres_port
host = postgres_host
db_name = postgres_db_name

In [16]:
# Create connection string
conn_string = f"postgresql://postgres:{password}@{host}:{port}/{db_name}"

engine = create_engine(conn_string)

In [17]:
# Confirm that the connection works
inspector = inspect(engine)

inspector.get_table_names()

['yellowstone', 'ufo_sightings']

#### Load UFO data into SQL database

In [18]:
ufos_wy_for_db.to_sql(name = "ufo_sightings", con = engine, if_exists = "append", index = False)

167

#### Load Yellowstone data into SQL database

In [ ]:
yellowstone_df.to_sql(name = "yellowstone", con = engine, if_exists = "append", index = False)